In [116]:
import pandas as pd
import numpy as np
import reverse_geocoder as rg
import matplotlib.pyplot as plt
from geopy.extra.rate_limiter import RateLimiter 

In [117]:
weather = pd.read_csv(r'..\Data\Clean\cweather.csv')

In [118]:
def stringchange(df, column = 'Location'):
    import re
    result = []
    for i in df[column]:
        sep = ' '
        result.append(sep.join(re.findall('[A-Z][^A-Z]*', i )))
    df[column] = result
    return df

In [119]:
weather = stringchange(weather)
weather = weather.replace({'Pearce R A A F': 'Pearce'})

In [120]:
#weather column Locations an eigene variable übergeben
weatherloc = weather.Location.unique()
weatherloc = pd.DataFrame(weatherloc, columns=['Location']) 

In [ ]:
#weatherloc = stringchange(weatherloc)
#weatherloc = weatherloc.replace({'Pearce R A A F': 'Pearce'})

In [122]:
np.all(weather.Location.unique() == weatherloc.Location.unique())

True

In [123]:
#applying geocoordinates to locations
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
gc = Nominatim(user_agent="fintu-blog-geocoding-python")

def rate_limited_geocode(query):
    geocode = RateLimiter(gc.geocode, min_delay_seconds=1)
    return geocode(query)
#geocode der 2ten funktion ist eine andere variable
def geocodeb(row):
    lookup_query = row["Location"] + ", " + "Australia"
    lookup_result = rate_limited_geocode(lookup_query)
    return lookup_result
weatherloc["loctemp"] = weatherloc.apply(geocodeb, axis=1)

In [125]:
# making sure that it finds values to all locations
weatherloc.isna().sum()

Location    0
loctemp     0
dtype: int64

In [126]:
tempbla = [i [1] for i in weatherloc.loctemp]
weatherloc['latitude'] , weatherloc['longitude'] = np.array(tempbla).T
weatherloc = weatherloc.drop(['loctemp'],1)

In [127]:
def reverse_filling(df1, df2):
    templatitude = np.zeros(df1.shape[0])
    templongitude = np.zeros(df1.shape[0])
    for i in df1.Location.unique():
        templatitude[df1.Location == i] = df2[df2.Location==i].latitude
        templongitude[df1.Location == i] = df2[df2.Location==i].longitude
    df1['latitude'] = templatitude
    df1['longitude'] = templongitude
    return df1

In [128]:
bla = reverse_filling(weather, weatherloc)

In [131]:
bla.latitude

0        -36.080477
1        -36.080477
2        -36.080477
3        -36.080477
4        -36.080477
            ...    
142188   -25.345554
142189   -25.345554
142190   -25.345554
142191   -25.345554
142192   -25.345554
Name: latitude, Length: 142193, dtype: float64